In [11]:
from PIL import Image
from io import BytesIO
import base64
import os

from byaldi import RAGMultiModalModel

In [12]:
with open('hf_key.txt', 'r') as file:
    hf_key = file.read().strip()

os.environ["HF_TOKEN"] = hf_key

In [ ]:
# Optionally, you can specify an `index_root`, which is where it'll save the index. It defaults to ".byaldi/".
RAG = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


In [ ]:
RAG.index(
    input_path="docs/", # The path to your documents
    index_name="test", # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=False, # Whether the index should store the base64 encoded documents.
    overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
)

NameError: name 'RAG' is not defined

In [ ]:
query = "What is Colpali?"

results = RAG.search(query, k=1)

In [ ]:
# Decode the Base64 image bytes
image_bytes = base64.b64decode(results[0].base64)

# Open the image from the bytes
image = Image.open(BytesIO(image_bytes))

# Show the image
image.show()